<a href="https://colab.research.google.com/github/chasubeen/TalesRunnner/blob/subeen/koT5_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# model 저장 경로 설정
data_path = "/content/drive/MyDrive/7th-project/data"
model_path = "/content/drive/MyDrive/7th-project/model/koT5"

# **1. 데이터 준비**


## **1-1. 데이터 불러오기**
- 전처리 된 csv 파일 불러오기
- 해당 csv 파일에는 다음과 같은 정보들을 포함하고 있음

| Field | Description |
| --- | --- |
| id |* 이미지 → 캡션 정보와 annotation 정보를 결합 |
| imgpath | * `동화 삽화 생성 데이터 > 라벨링 데이터` 내의 jpg 파일 |
| caption | * BLIP을 통과시켜 얻어진 캡션|
| srcText | * LM의 output text|
| name | * 객체의 일반명사 (필수)|
| i_action | * 객체의 행동, 상태 (필수) |
| classification | * 의사소통/자연탐구/사회관계/예술경험/신체운동_건강 (필수) |
| character | * 캐릭터 (선택) |
| setting | * 셋팅 (선택) |
| action | * 행동 (선택) |
| feeling | * 감정 (선택) |
| causalRelationship | * 인과 (선택) |
| outcomeResolution | * 결과 (선택) |
| prediction | * 예측 (선택) |

In [3]:
import pandas as pd

In [4]:
## 전처리된 데이터(csv 파일) 불러오기
# 파일 경로 수정 필요
train_data = pd.read_csv(f"{data_path}/train_sample.csv", encoding = "cp949")
val_data = pd.read_csv(f"{data_path}/val_sample.csv", encoding = "cp949")

print(f"Train Data: {len(train_data)} samples")
print(f"Validation Data: {len(val_data)} samples")

Train Data: 10 samples
Validation Data: 5 samples


In [5]:
train_data.head()

,id,caption,srcText,name,i_action,classification,character,setting,action,feeling,causalRelationship,outcomeResolution,prediction
0,1,숲속의 나비,나비는 숲속에서 춤추듯 날고 있었습니다.,나비,날다,자연탐구,아름다운 나비,숲속,춤추다,기쁨,숲속의 바람이 나비를 춤추게 했다.,숲속의 나비는 행복했다.,나비는 또 다른 꽃을 찾을 것이다.
1,2,바다의 고래,고래는 바다 속 깊은 곳을 유영하고 있었습니다.,고래,유영하다,자연탐구,거대한 고래,바다,NaN,평화,NaN,고래는 바다를 사랑하게 되었다.,NaN
2,3,하늘의 새,새는 푸른 하늘을 자유롭게 날고 있었습니다.,새,날다,자연탐구,NaN,하늘,날아오르다,NaN,하늘의 바람이 새를 떠오르게 했다.,NaN,새는 구름 위로 올라갈 것이다.
3,4,밤하늘의 별,별은 밤하늘에서 반짝이고 있었습니다.,별,반짝이다,예술경험,반짝이는 별,NaN,빛나다,감탄,NaN,별은 밤하늘에서 더 반짝이게 되었다.,NaN
4,5,초원의 사자,사자는 초원에서 고요히 쉬고 있었습니다.,사자,쉬다,자연탐구,NaN,초원,NaN,NaN,초원의 바람이 사자를 쉬게 했다.,사자는 초원에서 평온함을 느꼈다.,사자는 가족과 함께 쉴 것이다.


In [6]:
val_data.head()

,id,caption,srcText,name,i_action,classification,character,setting,action,feeling,causalRelationship,outcomeResolution,prediction
0,11,바람 속의 연,연은 바람 속에서 높이 날아오르고 있었습니다.,연,날다,자연탐구,높이 나는 연,바람,NaN,자유,바람이 연을 높이 날게 했다.,연은 더 높이 날아올랐다.,연은 더 멀리 날아갈 것이다.
1,12,숲속의 여우,여우는 숲속에서 조용히 움직이고 있었습니다.,여우,움직이다,자연탐구,조용한 여우,숲속,조용히 움직이다,NaN,NaN,여우는 숲에서 더 안전해졌다.,여우는 새로운 은신처를 찾을 것이다.
2,13,강물의 물고기,물고기는 강물 속에서 유유히 헤엄치고 있었습니다.,물고기,헤엄치다,자연탐구,NaN,강물,유유히 헤엄치다,평화,NaN,NaN,물고기는 강물 속에서 새로운 친구를 만날 것이다.
3,14,바다의 파도,파도는 바다에서 힘차게 밀려오고 있었습니다.,파도,밀려오다,자연탐구,NaN,바다,힘차게 밀려오다,NaN,바다의 바람이 파도를 밀어냈다.,NaN,NaN
4,15,하늘의 구름,구름은 하늘에서 천천히 흘러가고 있었습니다.,구름,흘러가다,자연탐구,NaN,하늘,NaN,평온,하늘의 바람이 구름을 흘러가게 했다.,구름은 하늘에 평화를 가져왔다.,NaN


## **1-2. input text 가공**
- special token을 활용하여 각 정보를 구분
  - 필수 필드는 결측치를 허용하지 않도록
  - 선택 필드는 결측치가 있는 경우 `<empty>` 토큰 부여
- caption과 보조적 정보를 포함하여 구성
  - 행별 고유 시드 설정: 각 행의 id(또는 index)를 seed로 설정   
  → 동일한 데이터를 사용하면 언제나 같은 결과가 나오도록 보장
  - 랜덤 순서 보장: 각 행마다 token의 순서를 랜덤하게 설정  
  → 모델이 token 간 순서나 위치에 지나치게 의존하지 않고, 각 요소의 의미와 역할을 학습하도록 유도




In [7]:
import random

In [8]:
### Input Text 생성 함수

def generate_input_text(row):
    """
    각 행(row) 데이터를 가공하여 특수 토큰을 포함한 input_text 생성.
    """

    ## 고유 seed 설정
    unique_id = row["id"] if "id" in row else row.name  # 'id'가 없으면 DataFrame index 사용
    random.seed(unique_id)


    ## 필수 필드
    # 존재성 보장(assertion)
    assert pd.notna(row['caption']) and row['caption'].strip(), "Error: 'caption' 필드는 비워둘 수 없습니다."
    assert pd.notna(row['name']) and row['name'].strip(), "Error: 'name' 필드는 비워둘 수 없습니다."
    assert pd.notna(row['i_action']) and row['i_action'].strip(), "Error: 'i_action' 필드는 비워둘 수 없습니다."
    assert pd.notna(row['classification']) and row['classification'].strip(), "Error: 'classification' 필드는 비워둘 수 없습니다."


    ## 필수 및 선택 필드 구성
    required_fields = [
        f"<caption> {row['caption']}",
        f"<name> {row['name']}",
        f"<i_action> {row['i_action']}",
        f"<classification> {row['classification']}"
    ]
    optional_fields = {
        "character": "<character>",
        "setting": "<setting>",
        "action": "<action>",
        "feeling": "<feeling>",
        "causalRelationship": "<causalRelationship>",
        "outcomeResolution": "<outcomeResolution>",
        "prediction": "<prediction>"
    }
    optional_tokens = [
        f"{token} {row[field]}" if pd.notna(row[field]) and row[field].strip() else f"{token} <empty>"\
        for field, token in optional_fields.items()
    ]


    ## 토큰 순서 섞기
    all_tokens = required_fields + optional_tokens
    random.shuffle(all_tokens)


    return " ".join(all_tokens)

In [9]:
## Input Text, Output Text 생성
train_data["input_text"] = train_data.apply(generate_input_text, axis = 1)
train_data["output_text"] = train_data["srcText"]

val_data["input_text"] = val_data.apply(generate_input_text, axis = 1)
val_data["output_text"] = val_data["srcText"]

In [10]:
train_data[["id", "input_text", "output_text"]].head()

,id,input_text,output_text
0,1,<action> 춤추다 <causalRelationship> 숲속의 바람이 나비를 ...,나비는 숲속에서 춤추듯 날고 있었습니다.
1,2,<feeling> 평화 <prediction> <empty> <classificat...,고래는 바다 속 깊은 곳을 유영하고 있었습니다.
2,3,<name> 새 <setting> 하늘 <action> 날아오르다 <caption>...,새는 푸른 하늘을 자유롭게 날고 있었습니다.
3,4,<i_action> 반짝이다 <setting> <empty> <feeling> 감탄...,별은 밤하늘에서 반짝이고 있었습니다.
4,5,<prediction> 사자는 가족과 함께 쉴 것이다. <i_action> 쉬다 <...,사자는 초원에서 고요히 쉬고 있었습니다.


In [11]:
val_data[["id", "input_text", "output_text"]].head()

,id,input_text,output_text
0,11,<caption> 바람 속의 연 <classification> 자연탐구 <i_act...,연은 바람 속에서 높이 날아오르고 있었습니다.
1,12,<outcomeResolution> 여우는 숲에서 더 안전해졌다. <predicti...,여우는 숲속에서 조용히 움직이고 있었습니다.
2,13,<feeling> 평화 <causalRelationship> <empty> <cap...,물고기는 강물 속에서 유유히 헤엄치고 있었습니다.
3,14,<prediction> <empty> <caption> 바다의 파도 <feeling...,파도는 바다에서 힘차게 밀려오고 있었습니다.
4,15,<setting> 하늘 <prediction> <empty> <i_action> 흘...,구름은 하늘에서 천천히 흘러가고 있었습니다.


# **2. Modeling**

In [12]:
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

## **2-1. model 준비**
- pre-trained `tokenizer`와 `model` 불러오기
- 한국어 말뭉치로 훈련된 koT5 활용
  - 현재는 **base** 활용
  - small version도 활용 가능

  ```python
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

  tokenizer = AutoTokenizer.from_pretrained("wisenut-nlp-team/KoT5-small")
  model = AutoModelForSeq2SeqLM.from_pretrained("wisenut-nlp-team/KoT5-small")
  ```

### **model file 구성**
1. **`config.json`**:
   - 모델 설정 파일
    - hidden size, layer 개수, attention heads 등
   - 모델 아키텍처와 관련된 정보를 포함

2. **`generation_config.json`**:
   - 텍스트 생성 시의 기본 설정값
    - `max_length`, `num_beams`, `repetition_penalty` 등

3. **`model.safetensors`**:
   - 모델의 가중치(weight)를 저장한 파일
    - `safetensors`는 더 안전하고 빠른 저장 형식을 사용

4. **`special_tokens_map.json`**:
   - 추가된 `special token`들의 정보를 명시적으로 저장
5. **`tokenizer_config.json`**:
   - `tokenizer`의 일반 설정
    - 사용된 vocabulary 크기, 특수 토큰 정보, tokenizer 클래스 이름 등을 저장
6. **`tokenizer.json`**:
   - `tokenizer`의 전체 단어 사전(vocabulary)과 매핑을 포함
   - 이 파일에 추가된 `special token`들이 반영되어, 새로운 토큰이 vocabulary에 추가된 것을 확인 가능

In [13]:
## 모델 및 Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained("wisenut-nlp-team/KoT5-base", revision="paraphrase")
model = AutoModelForSeq2SeqLM.from_pretrained("wisenut-nlp-team/KoT5-base", revision="paraphrase")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

## **2-2. Tokenizer 준비**
- special token 추가 및 model, tokenizer 확장

In [14]:
## 특수 token 정의
special_tokens = [
    "<caption>", "<name>", "<i_action>", "<classification>", "<character>",
    "<setting>", "<action>", "<feeling>", "<causalRelationship>",
    "<outcomeResolution>", "<prediction>", "<empty>"
]

In [15]:
## 특수 token 확장
# 1. 기존 특수 토큰 가져오기
existing_special_tokens = tokenizer.special_tokens_map.get("additional_special_tokens", [])

# 2. 특수 토큰 병합 (중복 제거)
merged_special_tokens = list(set(existing_special_tokens + special_tokens))

# 3. 토크나이저에 특수 토큰 추가
tokenizer.add_tokens(merged_special_tokens)

# 4. 모델 Embedding 크기 재조정
model.resize_token_embeddings(len(tokenizer))

# 5. 모델과 토크나이저 저장
tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

- 아래 두 파일들은 `tokenizer.save_pretrained()`로 업데이트되지 않기에 명시적으로 저장

In [16]:
## special_tokens_map.json 업데이트

special_tokens_map_path = f"{model_path}/special_tokens_map.json"

# 기존 special token 불러오기
with open(special_tokens_map_path, "r", encoding="utf-8") as f:
    special_tokens_map = json.load(f)

# 추가된 special token 반영
special_tokens_map["additional_special_tokens"] = merged_special_tokens

# token mapping update
with open(special_tokens_map_path, "w", encoding="utf-8") as f:
    json.dump(special_tokens_map, f, indent=4, ensure_ascii=False)

In [17]:
## tokenizer_config.json 업데이트

tokenizer_config_path = f"{model_path}/tokenizer_config.json"

# 기존 special token 불러오기
with open(tokenizer_config_path, "r", encoding="utf-8") as f:
    tokenizer_config = json.load(f)

# 추가된 special token 반영
tokenizer_config["additional_special_tokens"] = merged_special_tokens

# token mapping update
with open(tokenizer_config_path, "w", encoding="utf-8") as f:
    json.dump(tokenizer_config, f, indent=4, ensure_ascii=False)

## **2-3. Dataset 및 DataLoader 구성**
- model을 학습시키기 위한 데이터셋 구축 및 토큰화

In [18]:
from torch.utils.data import Dataset, DataLoader

In [19]:
### Dataset 클래스 정의

class StoryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length = 512):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_text = self.data.iloc[index]["input_text"]
        target_text = self.data.iloc[index]["output_text"]

        # 입력/출력 데이터 토큰화
        input_encoding = self.tokenizer(
            input_text,
            max_length = self.max_length,
            padding = "max_length",
            truncation = True,
            return_tensors = "pt",
        )
        target_encoding = self.tokenizer(
            target_text,
            max_length = self.max_length,
            padding = "max_length",
            truncation = True,
            return_tensors = "pt",
        )

        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_encoding["attention_mask"].squeeze(),
            "labels": target_encoding["input_ids"].squeeze(),
        }

In [20]:
## Dataset 생성
train_dataset = StoryDataset(train_data, tokenizer)
val_dataset = StoryDataset(val_data, tokenizer)

In [21]:
## DataLoader 생성
train_dataloader = DataLoader(train_dataset, batch_size = 4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size = 4)

# **3. Training**

In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

## **3-1. Configs**

In [24]:
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm
import torch
import evaluate

In [25]:
## 장치 설정 및 모델 확인

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [26]:
## Optimizer 및 Scheduler 설정

# optimizer 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# scheduler 설정
epochs = 10

num_training_steps = epochs * len(train_dataloader)
lr_scheduler = get_scheduler("linear", optimizer = optimizer, num_warmup_steps = 500,
                             num_training_steps = num_training_steps)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
## 평가 지표 설정
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

## **3-2. Run!!**

In [28]:
best_val_loss = float("inf")
patience = 3
early_stop_counter = 0

In [29]:
# torch cache 비우기

import gc

gc.collect()
torch.cuda.empty_cache()

In [30]:
### Training/Validation Loop

for epoch in range(epochs):
    print(f"\n=== Epoch {epoch + 1}/{epochs} ===")

    ## Training
    model.train()

    train_loss = 0

    for batch in tqdm(train_dataloader, desc = "Training"):
        optimizer.zero_grad()

        # Input
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # prediction
        outputs = model(input_ids = input_ids, attention_mask = attention_mask, labels = labels)

        # parameter update
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        train_loss += loss.item()

    # logging
    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Train Loss: {avg_train_loss:.4f}")


    ## Validation
    model.eval()

    val_loss = 0

    predictions, references = [], [] # prediction, GT

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc = "Validation"):
            # Input
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids = input_ids, attention_mask = attention_mask, labels = labels)

            val_loss += outputs.loss.item()

            # Prediction
            generated_ids = model.generate(
                input_ids = input_ids,
                attention_mask = attention_mask,
                max_length = 512,
                num_beams = 5,
                length_penalty = 1.0,
                no_repeat_ngram_size = 2,
                early_stopping = True,
            )

            # 디코딩 및 특수 토큰 제거
            batch_predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)
            batch_references = tokenizer.batch_decode(labels, skip_special_tokens = True)

            predictions.extend(batch_predictions)
            references.extend(batch_references)

    avg_val_loss = val_loss / len(val_dataloader)

    print(f"Validation Loss: {avg_val_loss:.4f}")


    ## 샘플 Predictions 및 References 출력
    print("\n=== Sample Predictions and References ===")
    for i in range(min(3, len(predictions))):  # 최대 3개의 샘플만 출력
        print(f"Prediction {i + 1}: {predictions[i]}")
        print(f"Reference {i + 1}: {references[i]}")


    ## Evaluation
    try:
        bleu_result = bleu.compute(predictions=predictions, references=[[r] for r in references])
        rouge_result = rouge.compute(predictions=predictions, references=references)
        print(f"BLEU Score: {bleu_result['bleu']:.4f}")
        print(f"ROUGE Scores: {rouge_result}")
    except Exception as e:
        print(f"Error during BLEU or ROUGE computation: {e}")


    ## Early Stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        model.save_pretrained(model_path)
        print("Best model saved.")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered.")
            break


=== Epoch 1/10 ===


Training:   0%|          | 0/3 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Train Loss: 30.1112


Validation:   0%|          | 0/2 [00:00<?, ?it/s]

...

Validation Loss: 29.0681

=== Sample Predictions and References ===
Prediction 1: 쿔의 바람은 연을 높이 날게 했다. 쌯은 바람 속의 연 닁큼의 바람 괐는지라 뫛 더 멀리 날아갈 것이다.
Reference 1: 연은 바람 속에서 높이 날아오르고 있었습니다.
Prediction 2: 쿔의 여우는 숲에서 더 안전해졌다. 쌯은 숲속의 여우 혙통탐구 폯의 숲속 닁큼 늭긴 숲속으로 괐는지라 뫧은 은신처를 찾으랴 뫨은은신처찾기 뫛?
Reference 2: 여우는 숲속에서 조용히 움직이고 있었습니다.
Prediction 3: 쌯이의 물고기는 강물 속에서 새로운 친구를 만날 것이다. 엕 즴의 평화 쳧의 즵의 물고기 뫛 강물에 잠긴 물고기를 만날 것입니다.
Reference 3: 물고기는 강물 속에서 유유히 헤엄치고 있었습니다.
BLEU Score: 0.0000
ROUGE Scores: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
Best model saved.

=== Epoch 10/10 ===


Training:   0%|          | 0/3 [00:00<?, ?it/s]

Train Loss: 22.1399


Validation:   0%|          | 0/2 [00:00<?, ?it/s]

Validation Loss: 26.8181

=== Sample Predictions and References ===
Prediction 1: 쿔의 바람은 연을 높이 날게 했다. 쌯은 바람 속의 연 닁큼의 바람 괐는지라 뫛 더 멀리 날아갈 것이다.
Reference 1: 연은 바람 속에서 높이 날아오르고 있었습니다.
Prediction 2: 쳧기의 여우는 숲에서 더 안전해졌다. 쌯이는 숲속의 여우 뫛? 혻기쯤에는 여우가 새로운 은신처를 찾을 것이다.
Reference 2: 여우는 숲속에서 조용히 움직이고 있었습니다.
Prediction 3: 쌯이의 물고기는 강물 속에서 새로운 친구를 만날 것이다. 엕 즴의 평화 쳧의 즵의 물고기 뫛 강물에 잠긴 물고기를 만날 것입니다.
Reference 3: 물고기는 강물 속에서 유유히 헤엄치고 있었습니다.
BLEU Score: 0.0000
ROUGE Scores: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
Best model saved.


# **4. Inference**

In [31]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import random

In [32]:
### 이야기 추론 함수

def generate_story(input_text, model, tokenizer, max_length=512, num_beams=5):
    ## 토큰화
    tokenized_text = tokenizer(
        input_text,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    ).to(model.device)


    ## 텍스트 생성
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=tokenized_text["input_ids"],
            attention_mask=tokenized_text["attention_mask"],
            max_length=max_length,
            num_beams=num_beams,
            repetition_penalty=2.0,
            length_penalty=1.2,
            no_repeat_ngram_size=2,
            early_stopping=True
        )


    ## 결과 디코딩
    generated_text = tokenizer.decode(
        generated_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )


    ## 불필요한 특수 토큰 제거
    special_tokens = tokenizer.special_tokens_map_extended["additional_special_tokens"]
    for token in special_tokens:
        generated_text = generated_text.replace(token, "")

    # 특수 토큰 및 불필요한 공백 제거
    generated_text = " ".join(generated_text.split())  # 중복 공백 제거

    return generated_text.strip()

In [33]:
if __name__ == "__main__":
    ## 모델 및 토크나이저 로드
    model_path = "/content/drive/MyDrive/7th-project/model/koT5"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


    ## GPU 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)


    ## 사용자 입력
    print("\n=== 동화 생성 ===")
    print("다음 정보를 입력하세요:")

    # 필수 정보
    caption = input("캡션 입력: ")
    name = input("이름(객체의 일반명사): ")
    i_action = input("행동 또는 상태: ")
    classification = input("분류(예: 자연탐구, 의사소통 등): ")

    # 선택적 정보
    character = input("캐릭터(선택): ") or "<empty>"
    setting = input("셋팅(선택): ") or "<empty>"
    action = input("행동(선택): ") or "<empty>"
    feeling = input("감정(선택): ") or "<empty>"
    causal_relationship = input("인과 관계(선택): ") or "<empty>"
    outcome_resolution = input("결과(선택): ") or "<empty>"
    prediction = input("예측(선택): ") or "<empty>"


    ## Input Text 생성
    input_text_tokens = [
        f"<caption> {caption}",
        f"<name> {name}",
        f"<i_action> {i_action}",
        f"<classification> {classification}",
        f"<character> {character}",
        f"<setting> {setting}",
        f"<action> {action}",
        f"<feeling> {feeling}",
        f"<causalRelationship> {causal_relationship}",
        f"<outcomeResolution> {outcome_resolution}",
        f"<prediction> {prediction}"
    ]
    random.seed(42)
    random.shuffle(input_text_tokens)
    input_text = " ".join(input_text_tokens)

    print(f"\n=== 입력 text ===\n{input_text}")


    ## 추론 실행
    generated_story = generate_story(input_text, model, tokenizer)
    print("\n=== 생성된 동화 ===")
    print(generated_story)


=== 동화 생성 ===
다음 정보를 입력하세요:
캡션 입력: 숲속의 나비
이름(객체의 일반명사): 나비
행동 또는 상태: 날다
분류(예: 자연탐구, 의사소통 등): 자연탐구
캐릭터(선택): 호랑
셋팅(선택): 하늘하늘
행동(선택): 날아가다
감정(선택): 즐겁다
인과 관계(선택): 
결과(선택): 
예측(선택): 

=== 입력 text ===
<feeling> 즐겁다 <classification> 자연탐구 <i_action> 날다 <causalRelationship> <empty> <setting> 하늘하늘 <action> 날아가다 <outcomeResolution> <empty> <character> 호랑 <caption> 숲속의 나비 <name> 나비 <prediction> <empty>

=== 생성된 동화 ===
쌯이는 즐겁다 혙통탐구 엕 뀯 뫛 괐는지라 쳧 뫖 괞찮을지라도 뫨은기러기떼의 숲속의 나비 즋 닁큼 혇태와 뎖은이는 나비의 날개 돖 먇의 하늘하늘 빐운 홇 뫧 뫏 릫뮯?
